In [31]:
#Imports
import csv
import os
import glob
import json
import re
import string
import pandas as pd
import numpy as np
import scipy
from scipy import constants
import seaborn as sns

#Matplotlib
import matplotlib.pyplot as plt

#SciKit
from sklearn.datasets import load_iris
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score

#MODELS:
#Naive Bayes Classifiers
from sklearn.naive_bayes import GaussianNB, MultinomialNB

#Support Vector Classifier
from sklearn.svm import LinearSVC

#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#NLTK
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer



In [32]:
#Extract features using count vectorizer
with open('./Tweet_Datasets/Classified_Tweets/Classified_Tweets.json', 'r', encoding='utf-8') as file:
   data = json.load(file)

#Extract TokenizedTweet values
tokenized_tweets = [entry["TokenizedTweet"] for entry in data]

#Create the CountVectorizer
cv = CountVectorizer(max_features=3000)
X_fin = cv.fit_transform([' '.join(inner_list) for inner_list in tokenized_tweets]).toarray()
feature_names = cv.get_feature_names_out()
bag_of_words_df = pd.DataFrame(X_fin, columns=feature_names)

#Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#Print the BOW matrix and export to CSV
#print("\nBOW Matrix:")
#print(bag_of_words_df)

#Export the BOW matrix to CSV
bag_of_words_df.to_csv('bag_of_words_matrix.csv', index=False)
print("\nBOW Matrix exported to 'bag_of_words_matrix.csv'")



BOW Matrix exported to 'bag_of_words_matrix.csv'


In [33]:
#Multinomial Naive Bayes Classifier
MultinomialNB = MultinomialNB()

#Gussian Naive Bayes Classifier
GaussianNB =  GaussianNB()

#Linear Support Vector Machine Classifier
LinearSVC = LinearSVC()

#Decision Tree Classifier
DecisionTreeClassifier = DecisionTreeClassifier()

In [34]:
with open('./Tweet_Datasets/Classified_Tweets/Classified_Tweets.json', 'r') as file:
   data = json.load(file)


classifications = [entry['Classification'] for entry in data]

X_train, X_test, y_train, y_test = train_test_split( 
                                               X_fin, 
                                               classifications, 
                                               test_size=0.2, 
                                                random_state=0)
MultinomialNB.fit(X_train, y_train)
GaussianNB.fit(X_train, y_train)
LinearSVC.fit(X_train, y_train)
DecisionTreeClassifier.fit(X_train, y_train)

/Users/joeyholzman/Desktop/CSAM_Machine_Learning/.venv/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


DecisionTreeClassifier()

In [35]:
#Multinomial Naive Bayes Results
MNBpred = MultinomialNB.predict(X_test)
print(classification_report(y_test, MNBpred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97        41
           1       0.94      1.00      0.97        29

    accuracy                           0.97        70
   macro avg       0.97      0.98      0.97        70
weighted avg       0.97      0.97      0.97        70



In [36]:
#Gaussian Naive Bayes Results
GNBpred = GaussianNB.predict(X_test)
print(classification_report(y_test, GNBpred))

              precision    recall  f1-score   support

           0       1.00      0.63      0.78        41
           1       0.66      1.00      0.79        29

    accuracy                           0.79        70
   macro avg       0.83      0.82      0.79        70
weighted avg       0.86      0.79      0.78        70



In [37]:
#Linear Support Vector Machine Classifier Results
SVMpred = LinearSVC.predict(X_test)
print(classification_report(y_test, SVMpred))


              precision    recall  f1-score   support

           0       0.98      0.98      0.98        41
           1       0.97      0.97      0.97        29

    accuracy                           0.97        70
   macro avg       0.97      0.97      0.97        70
weighted avg       0.97      0.97      0.97        70



In [38]:
#Decision Tree Classifier Results
DTpred = DecisionTreeClassifier.predict(X_test)
print(classification_report(y_test, DTpred))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94        41
           1       0.96      0.86      0.91        29

    accuracy                           0.93        70
   macro avg       0.94      0.92      0.93        70
weighted avg       0.93      0.93      0.93        70



In [ ]:
#PREDICTION CODE

In [41]:
nltk.download('stopwords')
nltk.download('wordnet')

def cleanText(tweet):
    # Replaces URLs with a special token
    tweet = re.sub(r'((www.\S+)|(https?://\S+))', '<URL>', tweet)
    # Removes the numbers from the text
    tweet = re.sub(r'[0-9]\S+', r'', tweet)
    # Removes tags from tweets except for hashtags
    tweet = re.sub(r'(@\S+)', r'', tweet)
    # Removes the punctuation from the text except for #
    tweet_without_punctuation = re.sub(r'[^#\w\s]', r'', tweet)
    # Sets stop words
    stop_words = set(stopwords.words("english"))
    # Removes the stop words from tweet
    tweet_without_stopwords = [word for word in tweet_without_punctuation.split() if word.lower() not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tweet_without_stopwords]
    # Join the list of cleaned words into a single string
    cleaned_tweet = ' '.join(lemmatized_words)
    return cleaned_tweet

def predict_tweet_classification(tweet, cv, MultinomialNB, GaussianNB, LinearSVC, DecisionTreeClassifier):
    # Preprocess the tweet
    cleaned_tweet = cleanText(tweet)
    # Vectorize the preprocessed tweet
    tweet_vectorized = cv.transform([cleaned_tweet])
    # Predict the classification using the trained models
    MNB_predicted_class = MultinomialNB.predict(tweet_vectorized)
    GNB_predicted_class = GaussianNB.predict(tweet_vectorized.toarray())
    SVM_predicted_class = LinearSVC.predict(tweet_vectorized)
    DT_predicted_class = DecisionTreeClassifier.predict(tweet_vectorized)
    
    # Create a dictionary containing all predicted classes
    predictions = {
        "MultinomialNB": MNB_predicted_class[0],
        "GaussianNB": GNB_predicted_class[0],
        "LinearSVC": SVM_predicted_class[0],
        "DecisionTreeClassifier": DT_predicted_class[0]
    }
    
    # Print the dictionary containing all predicted classes
    print(predictions)
    
    # Return the dictionary
    return predictions

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joeyholzman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/joeyholzman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [43]:
# Example usage
tweet = "#nude jailbait forums wet pussy cums https://t.co/YgNXenU44L"
# Assuming cv, MultinomialNB, GaussianNB, LinearSVC, and DecisionTreeClassifier are already initialized and trained
predicted_classes = predict_tweet_classification(tweet, cv, MultinomialNB, GaussianNB, LinearSVC, DecisionTreeClassifier)

{'MultinomialNB': 1, 'GaussianNB': 1, 'LinearSVC': 1, 'DecisionTreeClassifier': 1}
